In [1]:
import pandas as pd
import requests
import json

from datetime import datetime
import glob
from enum import Enum

from aind_data_schema.core.procedures import SpecimenProcedure, SpecimenProcedureType, Immunolabeling, HCRSeries


base = pd.read_excel("./ExM Sample Tracking.xlsx", sheet_name="ExM Tissue Processing", header=[0,1,2], converters={})


In [2]:
with pd.option_context("display.multi_sparse", False):
    print(base.head())
    

            priority                  #             Status      Processed By:  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
  Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2 Unnamed: 3_level_2   
0                NaN                1.0                NaN                NDO   
1                NaN                2.0                NaN                NDO   
2                NaN                3.0                NaN                NDO   
3                NaN                4.0                NaN                NDO   
4                NaN                5.0                NaN                NDO   

              Sample                                           Genotype  \
  Unnamed: 4_level_1                                 Unnamed: 5_level_1   
  Unnamed: 4_level_2                                 Unnamed: 5_level_2   
0             576404   Snap25-IRES2-Cre; Slc17a7-IRES2-FlpO-neo-5764...   
1             609105                               

In [3]:
subj_procedures = {}

download_files = False


if download_files:
    for row_idx, row in base.iterrows():    

        sample_id = str(row['Sample']['Unnamed: 4_level_1']['Unnamed: 4_level_2'])
        print(sample_id)
        
        if len(sample_id) != 6:
            continue

        # INFO: This originally loaded in the procedures files, but is not necessary at this moment since i have them saved. Prior block now loads these files into same structure.
        request = requests.get(f"http://aind-metadata-service/procedures/{sample_id}")
        #TODO: check for invalid/missing models

        if request.status_code == 404:
            print(f"{sample_id} model not found")

        item = request.json()
        print(item['message'])

        if item['message'] == 'Valid Model.':
            with open(f'./original_spec_files/{sample_id}.json', 'w') as outfile:
                json.dump(item['data'], outfile)
        else:
            with open(f'./original_spec_files/{sample_id}_invalid.json', 'w') as outfile:
                json.dump(item['data'], outfile)

In [4]:
class SpecMaker():
    def __init__(self, experimenter, sample_id, existing_procedure:dict=None) -> None:
        self.experimenter = experimenter
        self.sample_id = sample_id
        self.existing_procedure = existing_procedure
        
    def make_spec_procedure(self, start, end, procedure_type, procedure_name, notes=None, protocol_id="unknown", reagent=[], immunolabeling:Immunolabeling=None, hcr_series:HCRSeries=None):
        return SpecimenProcedure(
            procedure_type=procedure_type,
            procedure_name=procedure_name,
            specimen_id=self.sample_id,
            start_date=start,
            end_date=end,
            experimenter_full_name=self.experimenter,
            protocol_id=protocol_id,
            reagents=reagent,
            hcr_series=hcr_series,
            immunolabeling=immunolabeling,
            notes=notes,
        )
        

In [5]:


def basic_dict(name, start, end, type) -> dict:
        return {
            "procedure_name": name,
            "start_date": start,
            "end_date": end,
            "procedure_type": type,
        }

for row_idx, row in base.iterrows():

    items = []

    
    # general info
    processor = row['Processed By:']['Unnamed: 3_level_1']['Unnamed: 3_level_2']
    sample_id = str(row['Sample']['Unnamed: 4_level_1']['Unnamed: 4_level_2'])
    genotype = row['Genotype']['Unnamed: 5_level_1']['Unnamed: 5_level_2']
    label = row['Label']['Unnamed: 6_level_1']['Unnamed: 6_level_2']
    inj_age = row['Age of Injection']['Unnamed: 7_level_1']['Unnamed: 7_level_2']
    post_fix = row['Post-Fix']['Unnamed: 8_level_1']['Unnamed: 8_level_2']

    spec_maker = SpecMaker(processor, sample_id)

    # delipidation
    dcm_delip_name = 'DCM Delipidation (8-10d)'
    dcm_delip_start = row['DCM Delipidation (8-10d)']['Unnamed: 9_level_1']['Date Started']
    dcm_delip_end = row['DCM Delipidation (8-10d)']['Unnamed: 10_level_1']['Date Finished']
    items.append(basic_dict(dcm_delip_name, dcm_delip_start, dcm_delip_end, SpecimenProcedureType.DELIPIDATION))


    sbip_delip_name = 'SBiP Delipidation (7-9d)'
    sbip_delip_start = row['SBiP Delipidation (7-9d)']['Unnamed: 11_level_1']['Date Started']
    sbip_delip_end = row['SBiP Delipidation (7-9d)']['Unnamed: 12_level_1']['Date Finished']
    items.append(basic_dict(sbip_delip_name, sbip_delip_start, sbip_delip_end, SpecimenProcedureType.DELIPIDATION))

    # immunolabeling
    immunolabeling_name = 'Immunolabeling (31-33d)'
    immunolabeling_primary = row['Immunolabeling (31-33d)']['Unnamed: 13_level_1']['Primary (10ug, 20ug for primary Conjugate)']
    immunolabeling_secondary = row['Immunolabeling (31-33d)']['Unnamed: 14_level_1']['Secondary (20ug)']
    # items.append(basic_dict(immunolabeling_name, immunolabeling_primary, immunolabeling_secondary, SpecimenProcedureType.IMMUNOLABELING))

    # gelation
    mbs_gel_name = 'MBS Gelation'
    mbs_gel_start = row['Gelation (25d)']['MBS ']['Date Started']
    mbs_gel_end = row['Gelation (25d)']['MBS ']['Date Finished']
    items.append(basic_dict(mbs_gel_name, mbs_gel_start, mbs_gel_end, SpecimenProcedureType.GELATION))

    acx_gel_name = 'AcX Gelation'
    acx_gel_start = row['Gelation (25d)']['AcX']['Date Started']
    acx_gel_end = row['Gelation (25d)']['AcX']['Date Finished']
    items.append(basic_dict(acx_gel_name, acx_gel_start, acx_gel_end, SpecimenProcedureType.GELATION))

    pbs_gel_name = 'PBS Wash'
    pbs_gel_start = row['Gelation (25d)']['PBS Wash']['Date Started']
    pbs_gel_end = row['Gelation (25d)']['PBS Wash']['Date Finished']
    items.append(basic_dict(pbs_gel_name, pbs_gel_start, pbs_gel_end, SpecimenProcedureType.GELATION))

    stock_gel_name = 'Stock X + VA-044 Equilibration'
    stock_gel_start = row['Gelation (25d)']['Stock X + VA-044 Equilibration']['Date Started']
    stock_gel_end = row['Gelation (25d)']['Stock X + VA-044 Equilibration']['Date Finished']
    items.append(basic_dict(stock_gel_name, stock_gel_start, stock_gel_end, SpecimenProcedureType.GELATION))

    prok_gel_name = 'Gelation + ProK RT'
    prok_gel_start = row['Gelation (25d)']['Gelation +  ProK RT']['Date Started']
    prok_gel_end = row['Gelation (25d)']['Gelation +  ProK RT']['Date Finished']
    items.append(basic_dict(prok_gel_name, prok_gel_start, prok_gel_end, SpecimenProcedureType.GELATION))
    prok_rt_gel_time = row['Gelation (25d)']['Gelation +  ProK RT']['TIme']

    prok_37c_gel_name = 'Gelation + Add\'l ProK 37C'
    prok_37c_gel_start = row['Gelation (25d)']["Gelation + Add'l ProK 37C"]['Date Started']
    prok_37c_gel_end = row['Gelation (25d)']["Gelation + Add'l ProK 37C"]['Date Finished']
    items.append(basic_dict(prok_37c_gel_name, prok_37c_gel_start, prok_37c_gel_end, SpecimenProcedureType.GELATION))
    prok_37c_gel_time = row['Gelation (25d)']["Gelation + Add'l ProK 37C"]['Time']

    pbs2_gel_name = 'PBS Wash 2'
    pbs2_gel_start = row['Gelation (25d)']['PBS Wash']['Date Started.1']
    pbs2_gel_end = row['Gelation (25d)']['PBS Wash']['Date Finished.1']
    items.append(basic_dict(pbs2_gel_name, pbs2_gel_start, pbs2_gel_end, SpecimenProcedureType.GELATION))
    pbs_stored_bool = row['Gelation (25d)']['PBS Wash']['Stored in PBS Azide 0.01% at 4C']


    notes1 = row['Gelation (25d)']['PBS Wash']['Notes']
    notes2 = row['Gelation (25d)']['PBS Wash']['Notes.1']
    notes3 = row['Gelation (25d)']['PBS Wash']['Notes.2']


    specimen_procedures = []
    for item in items:
        print(item)
        if pd.isna(item['start_date']) or pd.isna(item['end_date']):
            continue
        specimen_procedures.append(spec_maker.make_spec_procedure(item['start_date'], item['end_date'], item['procedure_type'], item['procedure_name']))
    
    print(specimen_procedures)



{'procedure_name': 'DCM Delipidation (8-10d)', 'start_date': Timestamp('2021-10-01 00:00:00'), 'end_date': Timestamp('2021-10-11 00:00:00'), 'procedure_type': <SpecimenProcedureType.DELIPIDATION: 'Delipidation'>}
{'procedure_name': 'SBiP Delipidation (7-9d)', 'start_date': Timestamp('2021-10-25 00:00:00'), 'end_date': Timestamp('2021-11-03 00:00:00'), 'procedure_type': <SpecimenProcedureType.DELIPIDATION: 'Delipidation'>}
{'procedure_name': 'MBS Gelation', 'start_date': Timestamp('2021-11-10 00:00:00'), 'end_date': Timestamp('2021-11-11 00:00:00'), 'procedure_type': <SpecimenProcedureType.GELATION: 'Gelation'>}
{'procedure_name': 'AcX Gelation', 'start_date': Timestamp('2021-11-12 00:00:00'), 'end_date': Timestamp('2022-11-16 00:00:00'), 'procedure_type': <SpecimenProcedureType.GELATION: 'Gelation'>}
{'procedure_name': 'PBS Wash', 'start_date': datetime.datetime(2021, 11, 16, 0, 0), 'end_date': datetime.datetime(2021, 11, 18, 0, 0), 'procedure_type': <SpecimenProcedureType.GELATION: 'G

ValidationError: 1 validation error for SpecimenProcedure
start_date
  Input should be a valid date or datetime, invalid character in year [type=date_from_datetime_parsing, input_value='1/28/.2023', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/date_from_datetime_parsing